# Define Functions and Run Initial Query

In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import json

pd.options.display.float_format = '{:,.5f}'.format
pd.options.display.max_columns = 500
pd.options.display.max_rows = 20

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    TestGroup = np.array(('364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275')).astype(int)
    df['TestGroup'] = (df['Employee'].isin(TestGroup)).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    df['Proactive Launched']=(df['Date']>=datetime(2023,11,10)).astype(int)
    df['Constant'] =1
    return df

In [ ]:
vhe_dfp = run_query(
"""
select *
FROM 
    hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
    AND element_at(VHE.edp_raw_data_map, 'Scope')='GenerativeAISearchBotVoice'
    AND element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') in ('That worked!', 'Still need help', 'Wrong issue')
    AND TRY(CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) in (573190, 573276, 572909, 573192, 573585, 573573, 572815, 552121, 576911, 572247)
    OR CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) is null)
    AND DATE(edp_updated_date) > DATE('2023-11-10')
""")

In [ ]:
df = vhe_dfp.copy()
json_df = pd.DataFrame.from_records(df['edp_raw_data_map'].apply(json.loads))
# display(vhe_dfp.head(),json_df.head(),json_df.columns)
display(json_df[json_df['ExtraData_inputMessage']=='Wrong issue']['ExtraData_messageType'].head())
display(pd.DataFrame(json_df['ExtraData_inputMessage'].value_counts()).sort_values('ExtraData_inputMessage',ascending=False).head(20))
# json_df.to_excel('JsonData.xlsx')
del df, json_df


In [ ]:
home_df = run_query(
"""
SELECT
element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') AutoFeedback,
element_at(VHE.edp_raw_data_map, 'Name')
FROM 
    hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    --AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
    AND element_at(VHE.edp_raw_data_map, 'Scope')='GenerativeAISearchBotVoice'
    AND (element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') in ('That worked!', 'Still need help', 'Wrong issue') or element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') is null)
    AND (CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) in (573190, 573276, 572909, 573192, 573585, 573573, 572815, 552121, 576911, 572247)
    OR TRY CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) is null)
    AND DATE(edp_updated_date) > DATE('2023-11-10')
    
""")
home_df

In [ ]:
df_er=run_query(file_to_string(r'G:\Decision_Support\Reporting\Business Insights\Power BI\Generative AI Dashboards\Expert Assist Pilot\AdHocs\8. Finding Proactive Messages\ExpertAssistResponsesByType.sql'))


In [ ]:
df = df_er.copy()
df['Date']=pd.to_datetime(df['Date'])
df = df[df['Date']>='2023-11-10']
df = df[df['Date']<='2023-12-17']
df.sum()